In [ ]:
from pathlib import Path

from wys_ars.particles.ecosmog import Ecosmog
from wys_ars.simcoll import SimulationCollection as SimColl
from wys_ars.power_spectra.power_spectrum import PowerSpectrum

In [ ]:
dir_src = Path(__file__).parent.absolute()
sim_config_times = dir_src / "configs/particle_snapshot_info.h5"
sim_config_places = dir_src / f"configs/dtfe/cvg_{cosmo}_simulation_collection.yaml"

In [ ]:
domain_level = 512
amr_levels = (9, 9)

In [ ]:
# Initialise collection of simulations
simcoll = SimColl.from_file(sim_config_places, sim_config_times)

sim_nrs = {
    "sim1": [5],
    "sim2": [5],
    "sim3": [5],
    "sim4": [5],
    "sim5": [7],
}

# Loop over each simulation in the collection
for sim_name, sim in simcoll.sim.items():
    # Flatten AMR mesh into domain layer mesh using DTFE
    sim.dtfe(
        snap_nrs=sim_nrs[sim_name],
        file_root="snap_",
        quantities=["density_a", "velocity_a", "divergence_a"],
        file_dsc={
            "root": "snap_%03d." % sim_nrs[sim_name][0], 
            "extension": None,
        },
    )
    
    # Read Ramses cpu output binary files for a snapshot and convert into pandas DataFrame
    if "grav" in sim.file_dsc["root"]:
        fields = [
            "x",
            "y",
            "z",
            "phi",
            "f1",
            "f2",
            "f3",
        ]
    sim.compress_snapshot(
        amr_levels,
        domain_level,
        fields,
        sim.file_dsc["root"],
        None,
    )
    
    # Get power spectrum of all simulations
    pk = PowerSpectrum("particles", sim)
    file_dsc = {"root": "density_dtfe", "extension": "npy"}
    #file_dsc = {"root": "div_velocity_dtfe", "extension": "npy"}
    pk.compute(
        "dtfe_matter",
        #"div_velocity",
        None,
        file_dsc,
        None,
        save=True,
    )